Functions

In [11]:
from os.path import exists
import sys
from pickle import dump, load
import pickle

def load_cache(file):
    cache = pickle.load( open( file, "rb" ) )
    print(cache)
    return cache

def pickle_cache(variable, file_name):
    dump(variable, open(file_name, 'wb'))
    
def reset_all():
    global dictionary
    global no_lines
    global need_lines
    global bad_lines
    dictionary = {} 
    no_lines = []
    need_lines = []
    bad_lines = {} #{name:[list of bad lines]}
    
def see_all():
    print("Dictionary: ", len(dictionary))
    print("No_lines: ", len(no_lines))
    print("Need_lines: ", len(need_lines))
    print("Bad_lines: ", len(bad_lines))
    print(dictionary)
    print(no_lines)
    print(need_lines)
    print(bad_lines)
    
def load_all():
    global dictionary
    global no_lines
    global need_lines
    global bad_lines
    dictionary = load_cache("names_PUL_cache")
    no_lines = load_cache("names_no_lines_cache")
    need_lines = load_cache("names_need_lines_cache")
    bad_lines = load_cache("names_bad_lines_cache")

def pickle_all():
    global dictionary
    global no_lines
    global need_lines
    global bad_lines
    pickle_cache(dictionary, "names_PUL_cache")
    pickle_cache(no_lines, "names_no_lines_cache")
    pickle_cache(need_lines, "names_need_lines_cache")
    pickle_cache(bad_lines, "names_bad_lines_cache")

def pickle_all_by_date(date):
    global dictionary
    global no_lines
    global need_lines
    global bad_lines
    pickle_cache(dictionary, "names_PUL_cache"+date)
    pickle_cache(no_lines, "names_no_lines_cache"+date)
    pickle_cache(need_lines, "names_need_lines_cache"+date)
    pickle_cache(bad_lines, "names_bad_lines_cache"+date)

def recalculate_nolines_and_needlines(dictionary):
    global no_lines
    global need_lines
    no_lines = []
    need_lines = []
    for name, PULS in dictionary.items():
        if len(dictionary[name]) <= 3:
            need_lines.append(name)
        if len(dictionary[name]) <= 0:
            no_lines.append(name)

In [12]:
from datamuse import datamuse #may need to pip3 install python-datamuse
api = datamuse.Datamuse()
import praw #may need to pip install praw
import requests.auth
import string
reddit = praw.Reddit(client_id='fAkqHqn2XyBt3g',
                     client_secret="B1Rv2IY6KfA3PusqSeaFAu8brIw",
                     user_agent='USERAGENT')
import enchant #may need to pip install pyenchant
import itertools


def create_list_of_near_names(name):
    api = datamuse.Datamuse()
    list_of_dictionaries = api.words(sl = name, max = 10)
    list_of_names = []
    for i in list_of_dictionaries:
        if i['score'] == 100:
            list_of_names.append(i['word'])
    if name not in list_of_names:
        list_of_names.append(name)
    return list_of_names

def return_reddit_PULine(name):
    """Function that takes name and outputs comments from r/pickuplines that have the name in the title 
    and the name in the comment in a list. """
    pickuplines = []
    name = name.lower()
    for submission in reddit.subreddit('pickuplines').top(limit=20000):
        if " "+name in submission.title.lower():
            submission.comments.replace_more(limit=0)
            for top_level_comment in submission.comments:
                if name.lower() in top_level_comment.body.lower():
                    pickuplines.append(top_level_comment.body)

    for submission in reddit.subreddit('pickuplines').hot(limit=5000):
        if " "+name in submission.title.lower():
            submission.comments.replace_more(limit=0)
            for top_level_comment in submission.comments:
                if name.lower() in top_level_comment.body.lower():
                    pickuplines.append(top_level_comment.body)
    if not pickuplines:                
        for submission in reddit.subreddit('pickuplines').new(limit=10000):
            if " "+name in submission.title.lower():
                submission.comments.replace_more(limit=0)
                for top_level_comment in submission.comments:
                    if name.lower() in top_level_comment.body.lower():
                        pickuplines.append(top_level_comment.body)                    
                    
    if pickuplines:
        print(str(len(pickuplines))+" pick-up lines for "+ name.title()+ " found on Reddit.")
        return pickuplines
    else:
        print("No pickup lines for "+ name+ " found on Reddit.")
        return pickuplines

def new_reddit_PULines(name):
    """Function that takes name and outputs comments from r/pickuplines that have the name in the title 
    and the name in the comment in a list. """
    pickuplines = []
    name = name.lower() 
    for submission in reddit.subreddit('pickuplines').new(limit=10000):
        if " "+name in submission.title.lower():
            submission.comments.replace_more(limit=0)
            for top_level_comment in submission.comments:
                if name.lower() in top_level_comment.body.lower():
                    pickuplines.append(top_level_comment.body)                         
    if pickuplines:
        print(str(len(pickuplines))+" pick-up lines for "+ name.title()+ " found on Reddit.")
        return pickuplines
    else:
        print("No new pickup lines for "+name+ " found on Reddit.")
        return pickuplines
    
def return_palindrome_PULine(name):
    name = name.lower()
    i = 0
    j = len(name)-1
    while i<j:
        if name[i] != name[j]:
            print(name.title() + " is not a palindrome.")
            return []
        i = i+1
        j = j-1
    print(name.title() + " is a palindrome.")
    return [name.title() + " - your name is a palindrome, I like it. I'd do you forwards and backwards"]

def return_anagram_PULine_D(name):
    named = name.lower()+'d'
    english_dictionary = enchant.Dict("en_US")
    spanish_dictionary = enchant.Dict("es_ES")

    list_of_all_anagrams = ["".join(perm) for perm in itertools.permutations(named)]

    list_of_real_anagrams = []
    pickuplines = []
    spanish = False
    for i in list_of_all_anagrams:
        if english_dictionary.check(i):
            list_of_real_anagrams.append(i)

    if not list_of_real_anagrams:
        spanish = True
        for i in list_of_all_anagrams:
            if spanish_dictionary.check(i):
                list_of_real_anagrams.append(i)

    if not spanish:
        for i in list_of_real_anagrams:
            pickuplines.append(name.title() + ", if you add a D to your name, it's an anagram for the word "
                       + i
                       + ". So I guess the question is - do you want the D?")
            list_of_real_anagrams.remove(i)
    else:
        for i in list_of_real_anagrams:
            pickuplines.append(name.title() + ", if you add a D to your name, it's an anagram for the Spanish word "
                       + i
                       + ". So I guess the question is - do you want the D?")
            list_of_real_anagrams.remove(i)
    if pickuplines:
        print(name.title()+" has at least one anagram.")
    return pickuplines

def return_anagram_PULine_V(name):
    named = name.lower()+'v'
    english_dictionary = enchant.Dict("en_US")
    spanish_dictionary = enchant.Dict("es_ES")

    list_of_all_anagrams = ["".join(perm) for perm in itertools.permutations(named)]

    list_of_real_anagrams = []
    pickuplines = []
    spanish = False
    for i in list_of_all_anagrams:
        if english_dictionary.check(i):
            list_of_real_anagrams.append(i)

    if not list_of_real_anagrams:
        spanish = True
        for i in list_of_all_anagrams:
            if spanish_dictionary.check(i):
                list_of_real_anagrams.append(i)

    if not spanish:
        for i in list_of_real_anagrams:
            pickuplines.append(name.title() + ", if you add a V to your name, it's an anagram for the word "
                       + i
                       + ". So I guess the question is - do you want the V?")
            list_of_real_anagrams.remove(i)
    else:
        for i in list_of_real_anagrams:
            pickuplines.append(name.title() + ", if you add a V to your name, it's an anagram for the Spanish word "
                       + i
                       + ". So I guess the question is - do you want the V?")
            list_of_real_anagrams.remove(i)

    if pickuplines:
        print(name.title()+" has at least one anagram.")
        
    return pickuplines

def gather_all_PU_lines_for_a_name(name):
    names_list = create_list_of_near_names(name)
    PULines = []
    try:
        for each_name in names_list:
            PULines+=return_reddit_PULine(each_name)
        try:
            PULines+=return_palindrome_PULine(name)
            PULines+=return_anagram_PULine_D(name)
            PULines+=return_anagram_PULine_V(name)
        except:
            print("Failed palindrome/anagram line generation.")
            pass
    except:
        print("Failed Reddit Scrape.")
        pass
    print("Coming back from the interwebs...")
    return PULines

In [13]:
#Get common names from https://www.ssa.gov/cgi-bin/popularnames.cgi and run through https://wordtohtml.net/ 
import re
lines = '1MichaelJessica2MatthewAshley3ChristopherEmily4JacobSamantha5JoshuaSarah6NicholasTaylor7TylerHannah8BrandonBrittany9DanielAmanda10AustinElizabeth11AndrewKayla12JosephRachel13JohnMegan14ZacharyAlexis15RyanLauren16DavidStephanie17JamesCourtney18JustinJennifer19AnthonyNicole20WilliamVictoria21AlexanderBrianna22RobertAmber23JonathanMorgan24KyleDanielle25KevinJasmine26CodyAlexandra27ThomasAlyssa28ChristianRebecca29JordanMadison30AaronKatherine31BenjaminAnna32EricHaley33SamuelKelsey34DylanAllison35JoseMelissa36BrianAbigail37StevenKimberly38AdamShelby39NathanOlivia40TimothyMary41JasonMichelle42RichardKaitlyn43PatrickSydney44CharlesMaria45SeanChristina46JesseTiffany47AlexChelsea48JeremySara49CameronErin50JuanJordan51LoganNatalie52MarkBrooke53HunterMarissa54CalebHeather55ConnorAndrea56DakotaLaura57StephenMiranda58DevinPaige59LuisKatelyn60EvanSierra61TrevorGabrielle62DustinJulia63JaredVanessa64JeffreyKelly65GabrielKristen66TravisDestiny67IanEmma68CarlosSavannah69TaylorErica70BryanJacqueline71PaulMariah72KennethAlexandria73NathanielShannon74CoreyBriana75EthanBreanna76JesusAmy77BlakeKatie78BradleyCassandra79MitchellMadeline80GarrettCrystal81MarcusCaitlin82LukeLindsey83DerekKathryn84GregoryJenna85AntonioAngela86TannerCheyenne87DaltonMonica88EdwardMackenzie89PeterAlicia90VictorCatherine91LucasSelena92ScottCaroline93ElijahJamie94MiguelKaitlin95ChaseSabrina96SethBailey97AdrianGrace98IsaacMolly99NoahBrittney100ShawnAlexa'
lines = re.sub( r"([A-Z])", r" \1", lines)
for i in range(0,10):
    lines= lines.replace(str(i), ' ')
name_list = lines.split()

In [14]:
import numpy

def add_PUL_to_database(name, PUL):
    """Return updated dictionary"""
    if not name in dictionary:
        dictionary[name] = {PUL:15}
    else:
        if not PUL in dictionary[name]:
            dictionary[name][PUL] = 15
    try:
        if name in need_lines:
            if len(dictionary[name]) > 3:
                print("Removing "+name.title()+" from need_lines list.")
                need_lines.remove(name)
        if name in no_lines:
            if len(dictionary[name]) > 0:
                print("Removing "+name.title()+" from no_lines list.")
                no_lines.remove(name)
    except:
        print("Failed to remove '"+PUL+"' from "+ name.title()+" pick-up lines.")

            

def remove_PUL_from_database(name, PUL):
    try:
        del dictionary[name][PUL]
        print('Deleted '+name.title()+' pick-up line: ', PUL)
    except:
        print('Failed to delete '+name.title()+' pick-up line: ', PUL)
    try:
        if not name in need_lines:
            if len(dictionary[name]) <=3:
                print("Adding "+name.title()+" to need_lines list.")
                need_lines.append(name)
        if not name in no_lines:
            if len(dictionary[name]) <= 0:
                print("Adding "+name.title()+" to no_lines list.")
                no_lines.append(name)
    except:
        print("Failed to remove '"+PUL+"' from "+ name.title()+" pick-up lines.")    
            
def pick_PULs_from_database(dictionary_of_PULs):
    if len(dictionary_of_PULs)<=3:
        return list(dictionary_of_PULs.keys())
    else:
        lines = list(dictionary_of_PULs.keys())
        weights = []
        sum_of_weights = sum(dictionary_of_PULs.values())
        for i in dictionary_of_PULs.values():
            weights.append(i/sum_of_weights)
        chosen_PUL = list(numpy.random.choice(lines,size=3,replace=False, p=weights))
        return chosen_PUL
    
#pick_PULs_from_database({'hi':10, 'bye':10, 'lie':90, 'my': 2, 'guy': 100,})
def recieving_name_request(name):
    """Returns list of 0-3 pickup-lines. Adds new lines to database"""
    if name == '' or name == " ":
        return 'No Input.'
    name = name.lower()
    if name in no_lines:
        print(name.title()+" is in no_lines list.")
        return 'Give General.' #change
    else:
        if name in dictionary:
            print("Giving pick-up lines from database for "+name+".")
            return pick_PULs_from_database(dictionary[name]) 
        else:
            print(name.title()+" not in database. Searching online...")
            dictionary[name]={}
            names_PUL_from_internet = gather_all_PU_lines_for_a_name(name)
            if not names_PUL_from_internet:
                print("No lines found online for "+name.title()+". Adding to no_lines and need_lines list.")
                no_lines.append(name)
                need_lines.append(name)
                return 'Give General.'
            else:
                print("Found "+str(len(names_PUL_from_internet))+" pick-up lines online for "+name.title()+
                      ". Adding to database now.")
                for PUL in names_PUL_from_internet:
                    add_PUL_to_database(name, PUL)
                if len(dictionary[name]) <=3:#needs to be here
                    print("3 or less pick-up lines found for "+name.title()+
                         ". Adding it to the need_lines list.")
                    need_lines.append(name)    
                return pick_PULs_from_database(dictionary[name]) 

def update_name_from_internet(name):
    if name == '' or name == " ":
        return 'No Input.'
    name = name.lower()
    if not name in dictionary:
        recieving_name_request(name)
    else:
        print(name.title()+" is already in the database. It currently has "+str(len(dictionary[name]))+
              " pick-up lines. \n Scraping reddit for new lines...")
        names_PUL_from_internet = new_reddit_PULines(name)
        if names_PUL_from_internet:
            if not name in bad_lines:
                for PUL in names_PUL_from_internet:
                    add_PUL_to_database(name, PUL)             
            else:
                for PUL in names_PUL_from_internet:
                    if PUL in bad_lines[name]:
                        names_PUL_from_internet.remove(PUL)
                    else:
                        add_PUL_to_database(name, PUL)
            if names_PUL_from_internet:
                print("Adding ", names_PUL_from_internet, " to "+name+" pick-up lines. REPEATS WILL BE REMOVED.")
            else:
                print("No new pick-up lines found online for "+ name +".")

        else:
            print("No new pick-up lines found online for "+ name +".")

            

In [15]:
def recieving_weight_modifiers(name, PUL, value_modifier):
    if PUL in dictionary[name]:
        dictionary[name][PUL] += value_modifier
        if dictionary[name][PUL] <= 0:
            if name in bad_lines:
                if not PUL in bad_lines[name]:
                    bad_lines[name].append(PUL)
            else:
                bad_lines[name] = [PUL]
            try:
                del dictionary[name][PUL]
            except:
                print(PUL+" was already deleted.")
            if len(dictionary[name])<= 3:
                if not name in need_lines:
                    need_lines.append(name)
            if len(dictionary[name])<= 0:
                if not name in no_lines:
                    no_lines.append(name)


Giving pick-up lines from database for random.


['Is it hot in here or is it just you',
 'Girl, if I were a fly, I’d be all over you, because you’re the shit',
 'Roses are red violets are blue I didn’t know what perfect was until I met yo']

In [23]:
update_name_from_internet('libby')

Libby not in database. Searching online...
2 pick-up lines for Libby found on Reddit.
No pickup lines for libbie found on Reddit.
No pickup lines for libbey found on Reddit.
No pickup lines for libi found on Reddit.
No pickup lines for libye found on Reddit.
Libby is not a palindrome.
Coming back from the interwebs...
Found 2 pick-up lines online for Libby. Adding to database now.
3 or less pick-up lines found for Libby. Adding it to the need_lines list.


In [25]:
pickle_all()

In [26]:
see_all()

Dictionary:  223
No_lines:  98
Need_lines:  182
Bad_lines:  0
{'michael': {'2 turtles find themselves stuck in an oven. One turns to the other and shouts "wow! Michelle is hot!"': 15}, 'jessica': {}, 'matthew': {}, 'ashley': {'I can’t believe I Ashley matched with a girl as pretty as you ': 15, "Why don't you come over to my my place, and let's Sm-Ashley?": 15}, 'christopher': {}, 'emily': {"(Breathing heavily) Heh heh... can... can you dress up as... heh... Wheeze E-e-e-e-emily for me? Heh heh (Breathing heavily intensifies) She's mai waifu.": 15, '>lemme smash pls emily ': 15, 'I would choose you over Emilyion dollars.': 15, 'Emily only faked cancer once': 15, 'You are definitely one in Emily-llion': 15}, 'jacob': {}, 'samantha': {}, 'joshua': {"Joshua, if you add a D to your name, it's an anagram for the Spanish word hojudas. So I guess the question is - do you want the D?": 15}, 'sarah': {'If you could ride (or meet/see/touch/etc.) any dinosaur what would it be?\n\n*her answer*\n\n